In [3]:
import sqlite3


In [4]:
db = "/mnt/disk1/Github/total.db"

In [5]:
# fra chatGPT

def find(db, word):
    sql = f"""
WITH RECURSIVE bisection(lower, upper) AS (
  /* SELECT 0, COUNT(*) FROM words */
  select 0, 223039157
  UNION ALL
  SELECT lower, lower + (upper - lower) / 2
  FROM bisection WHERE (SELECT word FROM words
                        LIMIT 1 OFFSET lower + (upper - lower) / 2) < '{word}'
  UNION ALL
  SELECT lower + (upper - lower) / 2 + 1, upper
  FROM bisection WHERE (SELECT word FROM words
                        LIMIT 1 OFFSET lower + (upper - lower) / 2) >= '{word}'
)
SELECT word, freq FROM words
WHERE word = '{word}';
"""
    with sqlite3.connect(db) as con:
        res = con.execute(sql).fetchall()
    return res

def find2(db, word):
    sql = f"""
WITH RECURSIVE bisection(word, freq, lower, upper) AS (
  SELECT "", 0, min(rowid), max(rowid) FROM words
  UNION ALL
  SELECT word, freq, lower, lower + (upper - lower) / 2
  FROM bisection WHERE (SELECT word, freq FROM words
                        where rowid = lower + (upper - lower) / 2
                        and word < '{word}' limit 1) 
  UNION ALL
  SELECT word, freq, lower + (upper - lower) / 2 + 1, upper
  FROM bisection WHERE (SELECT word, freq FROM words
                          where rowid = lower + (upper - lower) / 2
                             and word >= '{word}' limit 1)
)
SELECT word, freq FROM bisection
WHERE word = '{word}';
"""
    with sqlite3.connect(db) as con:
        res = con.execute(sql).fetchall()
    return res

def simple(db, word):
    sql = f"select rowid, word, freq from words where word = '{word}' limit 1"
    with sqlite3.connect(db) as con:
        res = con.execute(sql).fetchall()
    return res

def query(db, sql, params=()):
    with sqlite3.connect(db) as con:
        res = con.execute(sql, params).fetchall()
    return res

In [128]:
def bisect(db, word):
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[0][0]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[0][0]
    found = False
    fromrow = minrow
    torow = maxrow
    targeted = ("",0)
    i = 0
    while (not found) :
        i += 1
        midrow = int(fromrow + (torow - fromrow)/2)
        targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[0]
        #print(midrow, targeted)
        if word == targeted[0] or torow == fromrow:
            #print(targeted, torow, fromrow)
            found = True
        elif word > targeted[0]:
            torow = midrow 
        else:
            fromrow = midrow
    print(i)
    return targeted

def bisectall(db, word):
    
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[0][0]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[0][0]
    
    def bisect(db, word, minrow, maxrow):
        found = False
        fromrow = minrow
        torow = maxrow
        targeted = ("",0)
        res = []
        i = 0
        delta = 0
        limit = math.log2(maxrow + 1)
        while (not found) and (fromrow <= torow) and i < limit:
            i += 1
            midrow = int(fromrow + (torow - fromrow)/2)
            targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[0]
            #print(midrow, targeted, torow, fromrow)
            if word == targeted[0]:
                res.append(targeted)
                print(targeted)
                found = True
                #print(i, targeted)
                left, li = bisect(db, word, fromrow, midrow -1)
                right, ri = bisect(db, word, midrow +1, torow )
                i = li + ri
                res += left
                res += right
            elif word > targeted[0]:
                torow = midrow
            else:
                fromrow = midrow 
            #print(i)
        return res, i
    
    return bisect(db, word, minrow, maxrow)


def bisectiterate(db, word):
    
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[0][0]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[0][0]
    
    def bisect(db, word, minrow, maxrow):
        found = False
        fromrow = minrow
        torow = maxrow
        targeted = ("",0)
        res = []
        i = 0
        delta = 0
        limit = math.log2(maxrow + 1)
        while (not found) and (fromrow <= torow) and i < limit:
            i += 1
            midrow = int(fromrow + (torow - fromrow)/2)
            targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[0]
            #print(midrow, targeted, torow, fromrow)
            if word == targeted[0]:
                res.append(targeted)
                #print(targeted)
                found = True
                #print(i, targeted, fromrow, torow)
                
                leftcandidate = midrow - 1
                lcandidate = query(db, f"select word, freq, rowid from words where rowid={leftcandidate}")[0]
                while lcandidate[0] == word and leftcandidate >= fromrow:
                    #print(leftcandidate)
                    res.append(lcandidate)
                    leftcandidate -= 1
                    lcandidate = query(db, f"select word, freq, rowid from words where rowid={leftcandidate}")[0]
                    
                rightcandidate = midrow + 1
                rcandidate = query(db, f"select word, freq, rowid from words where rowid={rightcandidate}")[0]
                while rcandidate[0] == word and rightcandidate <= torow:
                    #print(rightcandidate)
                    res.append(rcandidate)
                    rightcandidate += 1
                    rcandidate = query(db, f"select word, freq, rowid from words where rowid={rightcandidate}")[0]
                    
            elif word > targeted[0]:
                torow = midrow
            else:
                fromrow = midrow 
            #print(i)
        return res, i
    
    return bisect(db, word, minrow, maxrow)

In [126]:
%%time
bisect(db, "Nasjonalbiblioteket")

24
CPU times: user 9.4 ms, sys: 0 ns, total: 9.4 ms
Wall time: 8.14 ms


('Nasjonalbiblioteket', 419, 171961292)

In [133]:
%%time
bisectiterate(db, "håndterlig")

CPU times: user 5.5 ms, sys: 4.18 ms, total: 9.68 ms
Wall time: 8.18 ms


([('håndterlig', 315, 96661587), ('håndterlig', 747, 96661586)], 18)

In [130]:
%%time
bisectall(db, "fiskesuppe")

('fiskesuppe', 2256, 115322219)
('fiskesuppe', 8, 115322223)
('fiskesuppe', 715, 115322221)
('fiskesuppe', 1001, 115322220)
('fiskesuppe', 99, 115322222)
CPU times: user 10.4 ms, sys: 12.3 ms, total: 22.7 ms
Wall time: 21.1 ms


([('fiskesuppe', 2256, 115322219),
  ('fiskesuppe', 8, 115322223),
  ('fiskesuppe', 715, 115322221),
  ('fiskesuppe', 1001, 115322220),
  ('fiskesuppe', 99, 115322222)],
 54)

In [132]:
%%time
with sqlite3.connect(db) as con:
    print([x for x in con.execute("select word, freq, rowid from hyphendata where word = 'håndterlig'").fetchall()])

[('håndterlig', 315, 126501526), ('håndterlig', 747, 126437434)]
CPU times: user 0 ns, sys: 3.51 ms, total: 3.51 ms
Wall time: 2 ms


In [46]:
%%time
#find2(db, "spise")

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 14.5 µs


In [79]:
%%time
simple(db, "alvorlig")

CPU times: user 6.89 s, sys: 752 ms, total: 7.64 s
Wall time: 7.64 s


[(142016237, 'alvorlig', 307148)]

In [80]:
%%time
with sqlite3.connect(db) as con:
    print(con.execute("select * from words where rowid = 142016237 ").fetchall())

[('alvorlig', 307148)]
CPU times: user 1.85 ms, sys: 199 µs, total: 2.05 ms
Wall time: 1.22 ms


In [86]:
%%time
with sqlite3.connect(db) as con:
    print(con.execute("select * from hyphendata where rowid = 142016237 ").fetchall())

[('kultursplit', 'tet.', 'kultursplittet.', 1)]
CPU times: user 3.23 ms, sys: 0 ns, total: 3.23 ms
Wall time: 1.98 ms
